In [1]:
import torch
from meu_dataset import MeuDataset,avaliar_descritor,calcular_matching
from teste_util import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_dataset = "./data/datasets/features_path_flowers_dataset.pt"
# Carregar o dataset do arquivo "meu_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
#verificar se o objeto meu dataset está retornando o tensor correto
assert isinstance(meu_dataset,MeuDataset), 'o tipo de retorno não é MeuDataset'
assert isinstance(meu_dataset[0][0],torch.Tensor), 'o tipo de retorno não é torch.Tensor'


In [2]:
import gc
from torch.optim.lr_scheduler import ExponentialLR
gc.collect()
torch.cuda.empty_cache()
batch_size_siam = 50
from torch.utils.data import random_split, DataLoader


train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.4,0.4,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=False)

In [3]:
from tqdm import tqdm
import random

# Cosine similarity function
def cosine_similarity(a, b):
    a_norm = torch.nn.functional.normalize(a, dim=-1)
    b_norm = torch.nn.functional.normalize(b, dim=-1)
    return torch.mm(a_norm, b_norm.T)

def my_similarity(a, b):
    a_norm = torch.nn.functional.normalize(a, dim=-1)
    b_norm = torch.nn.functional.normalize(b, dim=-1)
    return torch.cdist(a_norm, b_norm, p=2)

# Triplet loss function
def triplet_loss(anchor, positive, negative, margin=0.2):
    similarities = cosine_similarity(anchor, positive)
    # Calcular a média da diagonal principal (âncoras vs. seus respectivos positivos)
    mean_diagonal = torch.mean(torch.diagonal(similarities))
    # Calcular a média dos outros elementos (âncoras vs. positivos não correspondentes)
    mean_other = torch.mean(similarities[~torch.eye(similarities.shape[0], dtype=torch.bool)])
    
    # losses = torch.relu(mean_other - mean_diagonal + margin)# losses considerando similaridade de cosseno
    losses = torch.relu((1 - mean_diagonal) + mean_other)# losses considerando similaridade de cosseno
    # losses = torch.relu(mean_diagonal - mean_other + margin)# my_similarity
    # print(losses,mean_diagonal,mean_other)
    return losses,mean_other,mean_diagonal


def train_one_epoch(model, data_loader, optimizer, loss_fn, device='cpu', is_training=True):
    model.train(is_training)
    total_loss = 0.

    progress_bar = tqdm(data_loader)
    for idx, data in enumerate(progress_bar):
        # Extract the anchor and positive batches
        anchor_batch, positive_batch = (
            data[0].to(device),
            data[1].to(device),
        )

        # Calculate descriptors for the anchor and positive images
        descs_anchor = model(anchor_batch)
        descs_pos = model(positive_batch)

        # Calculate the triplet loss
        loss,negativo,positivo = loss_fn(descs_anchor, descs_pos, None, margin=0.7)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        progress_bar.set_description(f'Loss: {loss.item()}-{negativo.item()}-{positivo.item()} - Total Loss: {total_loss/len(data_loader)}')

    return total_loss/len(data_loader)


In [45]:
from typing import List, Optional
import torch
from e2cnn import gspaces
from e2cnn import nn as enn    #the equivariant layer we need to build the model
import torch.nn.functional as F
from torch import nn
from typing_extensions import TypedDict

from kornia.core import Module, Tensor, concatenate
from kornia.filters import SpatialGradient
from kornia.geometry.transform import pyrdown
from kornia.utils.helpers import map_location_to_cpu

from kornia.feature.scale_space_detector import get_default_detector_config, MultiResolutionDetector,Detector_config


class KeyNet_conf(TypedDict):
    num_filters: int
    num_levels: int
    kernel_size: int
    Detector_conf: Detector_config


class _FeatureExtractor(Module):
    def __init__(self) -> None:
        super().__init__()

        self.hc_block = _HandcraftedBlock()
        self.lb_block = _LearnableBlock()

    def forward(self, x: Tensor) -> Tensor:
        x_hc = self.hc_block(x)
        x_lb = self.lb_block(x_hc)
        return x_lb


class _HandcraftedBlock(Module):
    def __init__(self) -> None:
        super().__init__()
        self.spatial_gradient = SpatialGradient('sobel', 1)

    def forward(self, x: Tensor) -> Tensor:
        sobel = self.spatial_gradient(x)
        dx, dy = sobel[:, :, 0, :, :], sobel[:, :, 1, :, :]

        sobel_dx = self.spatial_gradient(dx)
        dxx, dxy = sobel_dx[:, :, 0, :, :], sobel_dx[:, :, 1, :, :]

        sobel_dy = self.spatial_gradient(dy)
        dyy = sobel_dy[:, :, 1, :, :]

        hc_feats = concatenate([dx, dy, dx**2.0, dy**2.0, dx * dy, dxy, dxy**2.0, dxx, dyy, dxx * dyy], 1)

        return hc_feats


def _KeyNetConvBlock(
    feat_type_in,
    feat_type_out,
    r2_act,
    kernel_size: int = 5,
    stride: int = 1,
    padding: int = 2,
    dilation: int = 1,
) -> nn.Sequential:
    return enn.SequentialModule(
            enn.R2Conv(feat_type_in, feat_type_out, kernel_size=kernel_size, padding=padding, bias=False),
            enn.InnerBatchNorm(feat_type_out),
            enn.ReLU(feat_type_out, inplace=True),
        )


class _LearnableBlock(nn.Sequential):
    def __init__(self, in_channels: int = 10, out_channels: int = 8, group_size=8) -> None:
        super().__init__()
        r2_act = gspaces.Rot2dOnR2(N=group_size)

        feat_type_in = enn.FieldType(r2_act, in_channels * [r2_act.trivial_repr])
        self.in_type = feat_type_in
        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block0 = _KeyNetConvBlock(feat_type_in, feat_type_out, r2_act)

        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block1 = _KeyNetConvBlock(self.block0.out_type, feat_type_out, r2_act)

        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block2 = _KeyNetConvBlock(self.block1.out_type, feat_type_out, r2_act)
        self.gpool = enn.GroupPooling(self.block2.out_type)

    def forward(self, x: Tensor) -> Tensor:
        x = enn.GeometricTensor(x, self.in_type)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.gpool(x)
        # print("after gpool", x.shape)
        return x.tensor

class SoftHistogramLayer(nn.Module):
    def __init__(self, n_bin=5):
        super(SoftHistogramLayer, self).__init__()
        self.n_bin = n_bin

    def forward(self, x):
        batch_size, num_channels, height, width = x.shape
        out = x.view(batch_size, num_channels, -1)  # Redimensiona 'x' para um tensor 3D
        
        bin_edges = torch.linspace(0, 10, self.n_bin + 1, device=x.device)
        hist = []
        
        for i in range(batch_size):  # Itera sobre as instâncias no lote
            channel_hists = []
            
            for c in range(num_channels):  # Itera sobre os canais
                bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
                
                # Calcula o histograma suave para o canal 'c' da instância atual 'i'
                soft_hist = torch.exp(-(out[i, c, :, None] - bin_centers[None, :])**2 / 0.1).sum(dim=0)
                channel_hists.append(soft_hist)
            
            channel_hist_tensor = torch.stack(channel_hists)  # Empilha os histogramas suaves dos canais
            hist.append(channel_hist_tensor)
        
        hist = torch.stack(hist)  # Empilha os histogramas suaves das instâncias
        return hist.reshape(batch_size, -1)  # Redimensiona para um tensor 2D
    

class KeyNet(Module):
    def __init__(self, pretrained: bool = False, keynet_conf: KeyNet_conf = keynet_default_config) -> None:
        super().__init__()

        num_filters = keynet_conf['num_filters']
        self.num_levels = keynet_conf['num_levels']
        kernel_size = keynet_conf['kernel_size']
        padding = kernel_size // 2
        print("KeyNet config: ", keynet_conf)
        self.feature_extractor = _FeatureExtractor()
        
        self.last_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=num_filters * self.num_levels, out_channels=self.num_levels, kernel_size=kernel_size, padding=padding
            ),
            nn.ReLU(inplace=True),
        )
        self.softHistogramLayer = SoftHistogramLayer()
        if pretrained:
            print("KeyNet loaded")


    def forward(self, x: Tensor) -> Tensor:
        B, C, H, W = x.shape
        x= x.reshape(B*C, 1, H, W)
        shape_im = x.shape
        feats: List[Tensor] = [self.feature_extractor(x)]
        
        for i in range(1, self.num_levels):
            x = pyrdown(x, factor=1.2)
            feats_i = self.feature_extractor(x)
            feats_i = F.interpolate(feats_i, size=(shape_im[2], shape_im[3]), mode='bilinear')
            feats.append(feats_i)
        scores = self.last_conv(concatenate(feats, 1))
        # print("after pyramid", scores.shape)
        scores= scores.reshape(B,C*scores.shape[1],scores.shape[2],scores.shape[3])
        hist = self.softHistogramLayer(scores)
        # print("after hist", hist.shape)
        return hist
    
keynet_default_config: KeyNet_conf = {
    # Key.Net Model
    'num_filters': 8,
    'num_levels': 3,
    'kernel_size': 5,
    # Extraction Parameters
    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 20.0},
}

model =KeyNet().to(device)

KeyNet config:  {'num_filters': 8, 'num_levels': 3, 'kernel_size': 5, 'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 20.0}}


In [23]:
gc.collect()
torch.cuda.empty_cache()

def save_checkpoint(model, optimizer, epoch, loss, path):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss,
        # Adicione outras informações que você deseja salvar, como hiperparâmetros, configurações, etc.
    }
    torch.save(checkpoint, path)
    
def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    # Outras informações que você salvou no dicionário de checkpoint podem ser acessadas aqui
    return model, optimizer, epoch, loss


In [50]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epoch_i=-1
model_keynet = KeyNet().to(device)
optimizer = optim.Adam(model_keynet.parameters(), lr=0.001, weight_decay=0.0001)
PATH_MODEL = './data/models/feature_flowers_sp.pt'
model_keynet, optimizer, epoch_i, loss =load_checkpoint(model_keynet, optimizer,PATH_MODEL)
print("epoch_i ",epoch_i,"loss ",loss)

gc.collect()
torch.cuda.empty_cache()
epochs=300
i_epoch = 0
loss = 0

scheduler = ExponentialLR(optimizer, gamma=0.75)

KeyNet config:  {'num_filters': 8, 'num_levels': 3, 'kernel_size': 5, 'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 20.0}}
epoch_i  14 loss  0.5196015781164169


In [51]:
gc.collect()
torch.cuda.empty_cache()

def train_with_early_stopping(model, trainloader, testloader, criterion_d, optimizer, scheduler, device, epochs=100, patience=20):
    best_loss = float('inf')
    best_model = None
    epochs_without_improvement = 0

    for epoch in range(epoch_i+1,epochs):
        # Atualizar a taxa de aprendizado
        if (epoch % 16 == 0) and (epoch != 0):
            scheduler.step()
            
        running_loss = train_one_epoch(model, data_loader=trainloader, loss_fn=criterion_d,  optimizer=optimizer, device=device,is_training=True)

        with torch.no_grad():
            loss_test = train_one_epoch(model, data_loader=testloader, loss_fn=criterion_d,  optimizer=None, device=device,is_training=False)

        # Verificar se a perda melhorou
        if loss_test < best_loss:
            best_loss = loss_test
            epochs_without_improvement = 0
            best_model = model.state_dict()            
            save_checkpoint(model=model, epoch=epoch, optimizer=optimizer, loss=loss_test, path=PATH_MODEL)
            print("salvou no colab")
        else:
            epochs_without_improvement += 1

        # Verificar a condição de parada
        if epochs_without_improvement == patience:
            print(f"No improvement in loss for {epochs_without_improvement} epochs. Training stopped.")
            break

        print(f"Epoch [{epoch}/{epochs}] - Running Loss: {running_loss:.4f}, Test Loss: {loss_test:.4f}, Initial LR: {optimizer.param_groups[0]['initial_lr']:.6f}, Current LR: {optimizer.param_groups[0]['lr']:.6f}, Epochs without Improvement: {epochs_without_improvement}")

    # Carregar a melhor configuração do modelo
    model.load_state_dict(best_model)
    print(f'Epoch: {epoch}, Best Loss: {best_loss:.4f}')

train_with_early_stopping(model_keynet.to(device), dataloader_train, dataloader_val, triplet_loss, optimizer, scheduler, device, epochs=epochs, patience=100)

Loss: 0.5744235515594482-0.40661218762397766-0.8321886658668518 - Total Loss: 0.5069975882768631: 100%|██████████| 50/50 [01:01<00:00,  1.22s/it]  
Loss: 0.5607937574386597-0.32170647382736206-0.7609127163887024 - Total Loss: 0.5123343616724014: 100%|██████████| 50/50 [00:18<00:00,  2.64it/s]  


salvou no colab
Epoch [15/300] - Running Loss: 0.5070, Test Loss: 0.5123, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 0


Loss: 0.43879708647727966-0.3023764193058014-0.8635793328285217 - Total Loss: 0.4882033705711365: 100%|██████████| 50/50 [01:00<00:00,  1.21s/it]  
Loss: 0.5288327932357788-0.33927690982818604-0.8104441165924072 - Total Loss: 0.5172278141975403: 100%|██████████| 50/50 [00:18<00:00,  2.63it/s]  


Epoch [16/300] - Running Loss: 0.4882, Test Loss: 0.5172, Initial LR: 0.001000, Current LR: 0.000750, Epochs without Improvement: 1


Loss: 0.4607101380825043-0.30224713683128357-0.8415369987487793 - Total Loss: 0.47472964465618134: 100%|██████████| 50/50 [01:00<00:00,  1.22s/it] 
Loss: 0.528831958770752-0.3325652778148651-0.8037333488464355 - Total Loss: 0.5134063965082168: 100%|██████████| 50/50 [00:19<00:00,  2.59it/s]    


...

Loss: 0.41864514350891113-0.2885339856147766-0.8698888421058655 - Total Loss: 0.37177977919578553: 100%|██████████| 50/50 [01:00<00:00,  1.22s/it] 
Loss: 0.4887706935405731-0.32579347491264343-0.8370227813720703 - Total Loss: 0.5174092662334442: 100%|██████████| 50/50 [00:18<00:00,  2.63it/s] 


Epoch [62/300] - Running Loss: 0.3718, Test Loss: 0.5174, Initial LR: 0.001000, Current LR: 0.000422, Epochs without Improvement: 4


Loss: 0.3929004669189453-0.2859100103378296-0.8930095434188843 - Total Loss: 0.3784379452466965: 100%|██████████| 50/50 [01:00<00:00,  1.22s/it]   
Loss: 0.4951477646827698-0.32580626010894775-0.830658495426178 - Total Loss: 0.5213278865814209: 100%|██████████| 50/50 [00:18<00:00,  2.70it/s]   


Epoch [63/300] - Running Loss: 0.3784, Test Loss: 0.5213, Initial LR: 0.001000, Current LR: 0.000422, Epochs without Improvement: 5


Loss: 0.3872997760772705-0.28444093465805054-0.89714115858078 - Total Loss: 0.37404749989509584: 100%|██████████| 50/50 [00:59<00:00,  1.19s/it]   
Loss: 0.5283774137496948-0.32421448826789856-0.7958371043205261 - Total Loss: 0.08424505293369293:  16%|█▌        | 8/50 [00:02<00:15,  2.75it/s] 

In [ ]:
keynet_default_config: KeyNet_conf = {
    # Key.Net Model
    'num_filters': 8,
    'num_levels': 3,
    'kernel_size': 5,
    # Extraction Parameters
    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 20.0},
}

model =KeyNet(keynet_conf=keynet_default_config).to(device)
model_keynet, optimizer, epoch_i, loss =  load_checkpoint(model=model, optimizer=None, path=PATH_MODEL)

from meu_dataset import MeuDataset,avaliar_descritor
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.51)
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}  Errei: {total_erros}')

In [ ]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.025)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')

### Refazer o treinamento para fazer o descritor na imagem original ao inves da feature

In [ ]:
path_dataset = "./data/datasets/img_path_flowers_dataset.pt"
meu_dataset2 = MeuDataset.load_from_file(path_dataset)
train_dataset2, val_dataset2, test_dataset2 = random_split(meu_dataset2, [0.5,0.3,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train2 = DataLoader(train_dataset2, batch_size=batch_size_siam, shuffle=True)
dataloader_val2 = DataLoader(val_dataset2, batch_size=batch_size_siam, shuffle=True)
dataloader_test2 = DataLoader(test_dataset2, batch_size=batch_size_siam, shuffle=True)

In [ ]:
n_channel =1
model =Feature(n_channel=n_channel).to(device)
PATH_MODEL = './data/models/img_flowers_sp.pt'
model = train(model,dataloader_train2,dataloader_val2)

In [ ]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test2, model,th=0.2)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')
save_model(model, PATH_MODEL)

In [ ]:
from typing import List, Optional

import torch
import torch.nn.functional as F
from torch import nn
from typing_extensions import TypedDict

from kornia.core import Module, Tensor, concatenate
from kornia.filters import SpatialGradient
from kornia.geometry.transform import pyrdown
from kornia.utils.helpers import map_location_to_cpu

from kornia.feature.scale_space_detector import get_default_detector_config, MultiResolutionDetector,Detector_config


class KeyNet_conf(TypedDict):
    num_filters: int
    num_levels: int
    kernel_size: int
    Detector_conf: Detector_config


keynet_default_config: KeyNet_conf = {
    # Key.Net Model
    'num_filters': 8,
    'num_levels': 3,
    'kernel_size': 5,
    # Extraction Parameters
    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 20.0},
}


class _FeatureExtractor(Module):
    def __init__(self) -> None:
        super().__init__()

        self.hc_block = _HandcraftedBlock()
        self.lb_block = _LearnableBlock()

    def forward(self, x: Tensor) -> Tensor:
        x_hc = self.hc_block(x)
        x_lb = self.lb_block(x_hc)
        return x_lb


class _HandcraftedBlock(Module):
    def __init__(self) -> None:
        super().__init__()
        self.spatial_gradient = SpatialGradient('sobel', 1)

    def forward(self, x: Tensor) -> Tensor:
        sobel = self.spatial_gradient(x)
        dx, dy = sobel[:, :, 0, :, :], sobel[:, :, 1, :, :]

        sobel_dx = self.spatial_gradient(dx)
        dxx, dxy = sobel_dx[:, :, 0, :, :], sobel_dx[:, :, 1, :, :]

        sobel_dy = self.spatial_gradient(dy)
        dyy = sobel_dy[:, :, 1, :, :]

        hc_feats = concatenate([dx, dy, dx**2.0, dy**2.0, dx * dy, dxy, dxy**2.0, dxx, dyy, dxx * dyy], 1)

        return hc_feats


def _KeyNetConvBlock(
    feat_type_in,
    feat_type_out,
    r2_act,
    kernel_size: int = 5,
    stride: int = 1,
    padding: int = 2,
    dilation: int = 1,
) -> nn.Sequential:
    return enn.SequentialModule(
            enn.R2Conv(feat_type_in, feat_type_out, kernel_size=kernel_size, padding=padding, bias=False),
            enn.InnerBatchNorm(feat_type_out),
            enn.ReLU(feat_type_out, inplace=True),
        )


class _LearnableBlock(nn.Sequential):
    def __init__(self, in_channels: int = 10, out_channels: int = 8, group_size=8) -> None:
        super().__init__()
        r2_act = gspaces.Rot2dOnR2(N=group_size)

        feat_type_in = enn.FieldType(r2_act, in_channels * [r2_act.trivial_repr])
        self.in_type = feat_type_in
        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block0 = _KeyNetConvBlock(feat_type_in, feat_type_out, r2_act)

        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block1 = _KeyNetConvBlock(self.block0.out_type, feat_type_out, r2_act)

        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block2 = _KeyNetConvBlock(self.block1.out_type, feat_type_out, r2_act)
        self.gpool = enn.GroupPooling(self.block2.out_type)

    def forward(self, x: Tensor) -> Tensor:
        x = enn.GeometricTensor(x, self.in_type)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.gpool(x)
        return x.tensor

class SoftHistogramLayer(nn.Module):
    def __init__(self, n_bin=10):
        super(SoftHistogramLayer, self).__init__()
        self.n_bin = n_bin

    def forward(self, x):
        batch_size, num_channels, height, width = x.shape
        out = x.view(batch_size, num_channels, -1)  # Redimensiona 'x' para um tensor 3D
        
        bin_edges = torch.linspace(0, 10, self.n_bin + 1, device=x.device)
        hist = []
        
        for i in range(batch_size):  # Itera sobre as instâncias no lote
            channel_hists = []
            
            for c in range(num_channels):  # Itera sobre os canais
                bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
                
                # Calcula o histograma suave para o canal 'c' da instância atual 'i'
                soft_hist = torch.exp(-(out[i, c, :, None] - bin_centers[None, :])**2 / 0.1).sum(dim=0)
                channel_hists.append(soft_hist)
            
            channel_hist_tensor = torch.stack(channel_hists)  # Empilha os histogramas suaves dos canais
            hist.append(channel_hist_tensor)
        
        hist = torch.stack(hist)  # Empilha os histogramas suaves das instâncias
        return hist.reshape(batch_size, -1)  # Redimensiona para um tensor 2D
    

class KeyNet(Module):
    def __init__(self, pretrained: bool = False, keynet_conf: KeyNet_conf = keynet_default_config) -> None:
        super().__init__()

        num_filters = keynet_conf['num_filters']
        self.num_levels = keynet_conf['num_levels']
        kernel_size = keynet_conf['kernel_size']
        padding = kernel_size // 2

        self.feature_extractor = _FeatureExtractor()
        
        self.last_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=num_filters * self.num_levels, out_channels=num_filters, kernel_size=kernel_size, padding=padding
            ),
            nn.ReLU(inplace=True),
        )
        self.softHistogramLayer = SoftHistogramLayer()
        if pretrained:
            print("KeyNet loaded")


    def forward(self, x: Tensor) -> Tensor:
        shape_im = x.shape
        feats: List[Tensor] = [self.feature_extractor(x)]
        for i in range(1, self.num_levels):
            x = pyrdown(x, factor=1.2)
            feats_i = self.feature_extractor(x)
            feats_i = F.interpolate(feats_i, size=(shape_im[2], shape_im[3]), mode='bilinear')
            feats.append(feats_i)
        scores = self.last_conv(concatenate(feats, 1))
        print("scores ",scores.shape)
        return self.softHistogramLayer(scores)
    

n_channel =8
model =KeyNet().to(device)
image_a =torch.rand(40,n_channel,32,32).to(device)
image_p =torch.rand(40,n_channel,32,32).to(device)
B,C,H,W = image_a.shape
print("image_a ",image_a.shape, image_p.shape)
image_a =image_a.reshape(B*C,1,H,W)
image_p =image_p.reshape(B*C,1,H,W)
print("image_a ",image_a.shape, image_p.shape)
# Calculate descriptors for the anchor and positive images
descs_anchor = model(image_a)
descs_pos = model(image_p)

descs_anchor = descs_anchor.reshape(B,C,-1)
descs_pos = descs_pos.reshape(B,C,-1)
print("out ",descs_anchor.shape, descs_pos.shape)
from matplotlib import pyplot as plt

plt.imshow(image_a[0,0,:,:].cpu().detach().numpy())
plt.show()
print('descs_anchor ',descs_anchor[0])
# Calculate distances/similarities between anchor and all examples in the batch
distances = my_similarity(descs_anchor, descs_pos)  # Broadcasting
print("distances ",distances.shape)
# Choose the hardest negative example for each anchor
hard_negatives = torch.argmin(distances, dim=1)  # Get the index of the minimum similarity for each anchor

def similarity(desc1, desc2):
    desc1_norm = torch.nn.functional.normalize(desc1, dim=-1)
    desc2_norm = torch.nn.functional.normalize(desc2, dim=-1)
    return torch.sum(desc1_norm * desc2_norm)

print(descs_anchor.shape, descs_pos.shape)
similarity(descs_anchor, descs_pos)
